# Pitch shift

Сразу опишу проблему: нужно изменить частоту голоса на аудио не изменяя длительность записи. Идеально эту проблему решить невозможно т к если увеличиваешь частоту в 2 раза, уменьшается длинна волны в 2 раза и соответственно либо надо уменьшать длинну сигнала, либо запись проиграется дважды. Поэтому придумывают разные обходные пути.

## Library implementation

Это библеотечная имплементация: тут не совсем понятно как увеличить именно в 2 раза, но хотите повысить - ставьте n_steps > 0 (вроде для увеличения в 2 раза нужно поставить 4), понизить - n_steps < 0 (будет металлический звук, но без него невозможно)

In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft

/home/kn/.local/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
from IPython.display import Audio

In [3]:
Audio('peterpiper.wav')

In [4]:
y, sr = librosa.load('peterpiper.wav', sr=22050) 

In [5]:
y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=4) 

In [6]:
librosa.output.write_wav('new_auto.wav', y_shifted, sr)

In [7]:
Audio('new_auto.wav')

## Custom implementation

Это моя "имплементация": на stachexchange нашел что можно разбивать сэмпл на части а потом снова сшивать, это и делаю. 

chunks это кол-во частей, его лучше не ставить меньше 50 и больше 1000 

shift_rate это куда двигать: 2 - увеличить частоту в 2 раза, 0.5 - уменьшить в 2 раза, остальные значения будут работать хуже

Алгоритм таков:
- взять кусок семпла
- применить к нему fft (чтобы получить домейн амплитуда-от-частоты)
- сдвинуть частоты (если rate больше единицы то амплитуду частоты i записать на место амплитуды частоты rate * i, если меньше единицы то наоборот)
- сделать ifft (чтобы вернуться в первоначальный домейн)

Потом склеить все результаты в одно целое

In [8]:
def shift_pitch(y, shift_rate, chunks=100):
    y_res = np.zeros((len(y),), dtype=np.complex64)

    for i, piece in enumerate(np.array_split(y, chunks)):
        
        # приминяем fft
        piece_f = fft(piece)

        # место под новые частоты
        l = len(piece)
        temp = np.zeros((l,), dtype=np.complex64)

        # двигаем аплитуды
        if shift_rate > 1:
            for j in range(l // shift_rate):
                temp[shift_rate * j] = piece_f[j]
        else:
            rev_shift_rate = int(1 / shift_rate)

            for j in range(l // rev_shift_rate):
                temp[j] = piece_f[j * rev_shift_rate]

        # обратный fft
        piece_if = ifft(temp)

        # склевиаем все
        y_res[i * l: (i + 1) * l] = piece_if
        
    return y_res

## Examples

тоже работают не идеально

Оригинальное задание

In [9]:
y_res = shift_pitch(y, 2)

In [10]:
librosa.output.write_wav('new_inc_2.wav', np.asfortranarray(y_res.real), sr)

In [11]:
Audio('new_inc_2.wav')

Понижаем в 2 раза

In [12]:
y_res = shift_pitch(y, 0.5)

In [13]:
librosa.output.write_wav('new_dec_2.wav', np.asfortranarray(y_res.real), sr)

In [14]:
Audio('new_dec_2.wav')

Повысим в 4 раза (чтоб лучше работало поставлю кол-во чанков побольше)

In [15]:
y_res = shift_pitch(y, 4, 1000)

In [16]:
librosa.output.write_wav('new_inc_4.wav', np.asfortranarray(y_res.real), sr)

In [17]:
Audio('new_inc_4.wav')